In [8]:
import pandas as pd
import requests
import json
import numpy as np
from collections import Counter

def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'yes',
    from_date='2010-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams}&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&filter=has_ngrams:{has_ngrams}&page=1&per-page=20'
        ).json()
    
    a = []
    for x in range(30):
        try:
            y = institution['results'][x]['ngrams_url']
            if x < 5:
                print(institution['results'][x]['title'])
                print(institution['results'][x]['authorships'][0]['author']['display_name'])
                print(institution['results'][x]['doi'])
                print(institution['results'][x]['relevance_score'])
                print(institution['results'][x]['publication_date'])
                print(y,'\n')  
            z = requests.get(y).json()
            a.append(z)
        except:
            pass

    return a

_ngrams = searching('drought propagation',broad='no')

# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

How climate seasonality modifies drought duration and deficit
Anne Van Loon
https://doi.org/10.1002/2013jd020383
634.6617
2014-04-27
https://api.openalex.org/works/W2142552874/ngrams 

The propagation from meteorological to hydrological drought and its potential influence factors
Shengzhi Huang
https://doi.org/10.1016/j.jhydrol.2017.01.041
605.9497
2017-04-01
https://api.openalex.org/works/W2581646934/ngrams 

Evaluation of drought propagation in an ensemble mean of large-scale hydrological models
Anne Van Loon
https://doi.org/10.5194/hess-16-4057-2012
527.2203
2012-11-06
https://api.openalex.org/works/W2142313203/ngrams 

Understanding the Role of Climate Characteristics in Drought Propagation
Tushar Apurv
https://doi.org/10.1002/2017wr021445
521.2513
2017-11-01
https://api.openalex.org/works/W2766816780/ngrams 

Remote sensing for drought monitoring &amp; impact assessment: Progress, past challenges and future opportunities
Harry West
https://doi.org/10.1016/j.rse.2019.111291
477.390

In [9]:
ngram = []

for i in _ngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$',]

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]
df3.head(n=30)

110393


,ngram,ngram_count,ngram_tokens
3923,hydrological drought,645,2
3937,drought propagation,371,2
3917,meteorological drought,358,2
3919,soil moisture,278,2
3979,drought event,236,2
3951,drought characteristic,154,2
4120,time series,134,2
14588,scale model,118,2
3963,van loon,112,2
28834,accumulation period,106,2


In [10]:
queries = {}

for i in range(5):
   queries[f'0{i}'] = df3['ngram'].iloc[i]

# c = 0
# for i in queries:
#     c+=1

# print(c)

string = []
for idx,i in enumerate(queries):
    if idx == 0:
        string.append(
            f"{queries[i]}"
            f""
            )
    else:
        string.append(
            f"|"
            f"{queries[i]}"
            f""
            )
x = ''.join(string)
x

'hydrological drought|drought propagation|meteorological drought|soil moisture|drought event'

In [11]:
search = requests.get(
    f"https://api.openalex.org/works?"
    f"filter="
    f"title.search:{x}|"

    f"&sort=relevance_score:desc&page=1&per-page=20"
).json()

resultlist = []

for x in range(10):
    result = {}
    for y in search['results'][x].keys():
        result[y] = search['results'][x][y]
    resultlist.append(result)

x = [i['title'] for i in resultlist]
y = [i['authorships'][0]['author']['display_name'] for i in resultlist]
z = [i['doi'] for i in resultlist]
zz = [i['relevance_score'] for i in resultlist]

for idx, i in enumerate(x):
    print(i)
    print(y[idx])
    print(z[idx])
    print(zz[idx])
    print('\n')

Investigating soil moisture–climate interactions in a changing climate: A review
Sonia I. Seneviratne
https://doi.org/10.1016/j.earscirev.2010.02.004
2308.4438


The Soil Moisture Active Passive (SMAP) Mission
Dara Entekhabi
https://doi.org/10.1109/jproc.2010.2043918
2275.3674


Regions of Strong Coupling Between Soil Moisture and Precipitation
Randal D. Koster
https://doi.org/10.1126/science.1100217
2165.2466


Soil moisture retrieval from space: the Soil Moisture and Ocean Salinity (SMOS) mission
Yann Kerr
https://doi.org/10.1109/36.942551
2043.0154


Soil moisture retrieval from AMSR-E
Eni G. Njoku
https://doi.org/10.1109/tgrs.2002.808243
1729.9875


Hydrological drought explained
Anne Van Loon
https://doi.org/10.1002/wat2.1085
1643.5631


A Statistical Exploration of the Relationships of Soil Moisture Characteristics to the Physical Properties of Soils
Bernard J. Cosby
https://doi.org/10.1029/wr020i006p00682
1560.1328


Measuring soil moisture with imaging radars
P. Dubois
https://